In [1]:
import numpy as np, random, operator, pandas as pd

import networkx as nx

In [2]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance

    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"


In [3]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0

    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness


In [4]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [5]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

In [6]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)


In [7]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()

    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [8]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [9]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []

    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])

    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child


In [10]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])

    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [11]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))

            city1 = individual[swapped]
            city2 = individual[swapWith]

            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [12]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []

    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [13]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration


In [14]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations,rewiring):
    pop = initialPopulation(popSize, population)
    #initial watts and strogatz network
    G = nx.generators.random_graphs.barabasi_albert_graph(popSize,3,0)

    scores = []

    for l in range(0,generations):
        gen_scores=[]
        for j in range(0, popSize):
            candidato = pop[j]
            vecinos=[]
            for k in G[j]:
                vecinos.append(k)

            if(len(vecinos)>0):
                #Select best neighbour
                mejor_vecino = 0
                puntaje_mejor_vecino = 100000000

                for k in G[j]:
                    fitness_vecino = 1/Fitness(pop[k]).routeFitness()
                    if(fitness_vecino<puntaje_mejor_vecino):
                        puntaje_mejor_vecino = fitness_vecino
                        mejor_vecino = k

                if(puntaje_mejor_vecino<100000000):
                    pareja = pop[mejor_vecino]
                    child = breed(candidato, pareja)
                    child = mutate(child,mutationRate)

                    score_individuo  = 1/Fitness(candidato).routeFitness()
                    score_child = 1/Fitness(child).routeFitness()

                    if(score_child < score_individuo):
                        pop[j] =  child
                        G.add_edge(j,mejor_vecino)
                        gen_scores.append(score_child)
                    else:
                        gen_scores.append(score_individuo)

        for j in range(0, popSize):
            aristas = G.adj[j]

            vecinos=[]
            for k in aristas:
                vecinos.append(k)

            for i in range(0,len(vecinos)):
                if random.random() < rewiring:
                    random_index = int(random.random()*100)
                    G.remove_edge(j, vecinos[i])
                    G.add_edge(j,random_index)

        gen_best = min(gen_scores)
        scores.append(gen_best)

    return scores

In [15]:
cityList = []

#25 cities
for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 50
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 50

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


Print averages per iteration:
1860.575946665
1756.467622938
1691.938790621
1618.879722438
1563.999935705
1517.601604329
1471.492137441
1442.882231557
1407.090279455
1382.182889231
1359.369795322
1333.493038196
1306.000711528
1283.457569812
1269.226795940
1255.068802819
1232.570058157
1213.028913636
1203.893500470
1187.512745597
1181.762431490
1167.642407133
1156.527085723
1147.991876248
1137.035870870
1124.555078440
1114.930818271
1110.055582787
1104.301088933
1093.421888902
1088.231305253
1080.694440748
1074.993789833
1066.370715883
1058.427763375
1049.488613053
1043.264878272
1042.594077674
1035.682035041
1032.657604819
1027.359524111
1022.213548698
1013.310415300
1005.752804477
1001.206600559
996.504862793
991.956309372
987.945820140
985.918128257
984.530105196


In [16]:
cityList = []

#25 cities
for i in range(0,50):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 200

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


Print averages per iteration:
4302.482770072
4158.383511013
4035.774274526
3932.851288064
3836.041000186
3745.367804380
3666.274770393
3602.052596152
3541.142699062
3481.064452974
3421.889739434
3376.631614605
3340.013506046
3286.916340281
3253.000171299
3214.792898771
3179.461674972
3147.598291967
3113.998